# Assignment 1

## Preprocessing

In [ ]:
# Remove <>


The task:

1. Loop over the each text in the in folder
2. Extract the following information
    - Relative frequenct of Nouns, Verbs, Adjective and Adverbs per 10.000 words
    - total number of unique PER, LOC, ORGS
3. for each sub-folder (a1,a2,a3,..) save a table which shows the following information

In [3]:
import os

In [ ]:
#1. loading 
#- loop over subfolder
    #loop over each file
#2. nlp spacy
#3. save results

In [14]:
#Use os.listdir() to print all files (we want to work with).

os.listdir("/work/IdaHeleneDencker#2808/CDS_language/CDS_lang24/cds-lang-assignment-1/in/USEcorpus")

datapath=("/work/IdaHeleneDencker#2808/CDS_language/CDS_lang24/cds-lang-assignment-1/in/USEcorpus")

dirs= sorted(os.listdir(data_path))
for directory in dirs:
    subfolder= data_path + directory
    filenames=  sorted(os.listdir(subfolder))
    for text_file in filenames:
        print(subfolder + "/"+text_file)
        #load file
        #process with spacy
        #save results 
  



files = [f for f in os.listdir("/work/IdaHeleneDencker#2808/CDS_language/CDS_lang24/cds-lang-assignment-1/in/USEcorpus") if os.path.isfile(f)]



#filename = (".." ,"in","USEcorpus")

#with open(filename, encoding = "latin-1") as f:
    #text= f.read()


#for file in folder:

In [15]:
files

[]

In [ ]:
#iterate over data 

In [13]:
files

[]

In [2]:
filename

('..', 'in', 'USEcorpus')

In [ ]:
# load data 
filename= "../in/cds-assignment-templates/cds-lang-assignment-1"

with open(filename, encoding = "latin-1") as f:
    text= f.read()

In [ ]:
#make for-loop that loads all the files in 